In [ ]:
pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1') #совсем плохо
#model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2') # плохо
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2') #плохо

In [ ]:
#Sentences we want to encode. Example:
with open('csl_black.txt', 'r', encoding='utf8') as f:
    cslav_black = f.read().split('#')

with open('greek_black.txt', 'r', encoding='utf8') as n:
    greek_black = n.read().split('#')

In [ ]:
print(len(cslav_black), len(greek_black))

58 58


In [ ]:
import re

In [ ]:
cslav_black_no_n = []
for elem in cslav_black:
    cleaned_elem = re.sub(r'\n', '', elem)
    cslav_black_no_n.append(cleaned_elem)

In [ ]:
greek_black_no_n = []
for elem in greek_black:
    cleaned_elem = re.sub(r'\n', '', elem)
    greek_black_no_n.append(cleaned_elem)

In [ ]:
print(cslav_black_no_n[:5])
print('------------')
print(greek_black_no_n[:5])

['', "1Воскресние Твое Христе Спасе, Ангели поют на небесех, и нас на земли сподоби чистым сердцем Тебе славити. | Слава Святей, и Единосущней, и Животворящей, и Неразделимей Троице, всегда, ныне и присно, и во веки веков.Аминь.Христос воскресе из мертвых, смертию смерть поправ, и сущым во гробех живот даровав. | Да воскреснет Бог, и расточатся врази Его, и да бежат от Лица Его ненавидящии Его.Христос воскресе: | Яко исчезает дым, да исчезнут, яко тает воск от лица огня. Христос воскресе: | Тако да погибнут грешницы от Лица Божия, а праведницы да возвеселятся. Христос воскресе: | Сей день егоже сотвори Господь, возрадуемся и возвеселимся вонь. Христос воскресе: | Христос воскресе: | Христос воскресе: | Христос воскресе из мертвых, смертию смерть поправ. | И братиям поющым: И сущым во гробех живот даровав. | Миром Господу помолимся:Яко подобает Тебе всякая слава:Воскресния день: Ирмосы на 4:Христос воскресе из мертвых.Воскресния день: | Пасхальный канон поется полностью хором: и ирмосы,

In [ ]:
em_1 = model.encode(cslav_black_no_n) #цсл
em_2 = model.encode(greek_black_no_n) #греч

In [ ]:
cosine_scores = util.cos_sim(em_2, em_1) 
#порядок - это то, с чем сравниваемся - в моем случае греч с цсл

In [ ]:
print(type(cosine_scores))

<class 'torch.Tensor'>


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'Greek':[],
                   'CSL_1':[], 'CSL_1_Index':[], 
                   'CSL_2':[], 'CSL_2_Index':[], 
                   'CSL_3':[], 'CSL_3_Index':[],  
                   'CSL_4':[], 'CSL_4_Index':[],  
                   'CSL_5':[], 'CSL_5_Index':[], })

In [ ]:
df

,Greek,CSL_1,CSL_1_Index,CSL_2,CSL_2_Index,CSL_3,CSL_3_Index,CSL_4,CSL_4_Index,CSL_5,CSL_5_Index


In [ ]:
for i in range(len(greek_black_no_n)):
    results = []
    for n in range(len(cslav_black_no_n)):
        result = cosine_scores[i][n].item()
        results.append(result)

    max_results = sorted(results, reverse = True)[:10] #предположим, что минимальные отловятся в 100
    
    csl_all_indexes = [] #все топовые индексы
    for elem in max_results:
        csl_all_indexes.append(results.index(elem)) #индексы лучших результатов в цсл
    
    csl_indexes = csl_all_indexes[:5]
    
    df = df.append({'Greek':greek_black_no_n[i],
                    'CSL_1': cslav_black_no_n[csl_indexes[0]], 'CSL_1_Index':csl_indexes[0], 
                    'CSL_2':cslav_black_no_n[csl_indexes[1]], 'CSL_2_Index':csl_indexes[1],
                    'CSL_3':cslav_black_no_n[csl_indexes[2]], 'CSL_3_Index':csl_indexes[2],
                    'CSL_4':cslav_black_no_n[csl_indexes[3]], 'CSL_4_Index':csl_indexes[3],
                    'CSL_5':cslav_black_no_n[csl_indexes[4]], 'CSL_5_Index':csl_indexes[4]}, ignore_index=True)

In [ ]:
df

In [ ]:
df.to_csv('all-MiniLM-L6-v2.csv')